In [181]:
import csv
import requests
import json
import random

In [185]:
#finding the optimal weights and MAP score of the Vector Space Retreival model
#for a single query on Cranfield Dataset
def search(query):
    relevant_docs = []
    with open(r'C:\Users\NEW\Downloads\cran\cranqrel', 'r') as rel:
        csv_read = csv.reader(rel)
        for row in csv_read:
            q_no, doc_id, rel_no = row[0].split()
            if int(q_no) == 2:
                if int(rel_no)>=2:
                    relevant_docs.append(doc_id)
    print(relevant_docs) 
    
    precision_array = []
    map_array = []
    for a in range(10):
        for b in range(10):
            title = query['query']['multi_match']['fields'][0].split('^')
            title[1] = str(a)
            title = '^'.join(title)
            query['query']['multi_match']['fields'][0] = title
            data = query['query']['multi_match']['fields'][1].split('^')
            data[1] = str(b)
            data = '^'.join(data)
            query['query']['multi_match']['fields'][1] = data
                     
            url = 'http://localhost:9200/cran_experiment/_search'
            httpResp = requests.get(url, data=json.dumps(query), headers={"content-type":"application/json"})
            searchHits = json.loads(httpResp.text)["hits"]
            rel_docs = 0
            c = 0
            precision = []
            for idx, hit in enumerate(searchHits['hits']):
                if hit['_source']['id'] in relevant_docs:
                    rel_docs += 1
                c += 1
                precision.append(rel_docs/c)
        precision_array.append([sum(precision), a, b])
            
    map_array = [x[0]/10 for x in precision_array]
    print(len(map_array))
    print(f"The MAP Score of the corpus on the given query is: {max(map_array)}")
    
    print(f"The optimized weights are: ")
    for i in range(len(map_array)):
        if map_array[i] == max(map_array):
            a = precision_array[i][1]
            b = precision_array[i][2]
            title = query['query']['multi_match']['fields'][0].split('^')
            title[1] = str(a)
            title = '^'.join(title)
            query['query']['multi_match']['fields'][0] = title
            data = query['query']['multi_match']['fields'][1].split('^')
            data[1] = str(b)
            data = '^'.join(data)
            query['query']['multi_match']['fields'][1] = data
            url = 'http://localhost:9200/cran_experiment/_search'
            httpResp = requests.get(url, data=json.dumps(query), headers={"content-type":"application/json"})
            searchHits = json.loads(httpResp.text)["hits"]
            print ("Num\tRelevance Score\t\t ID\t Title")
            for idx, hit in enumerate(searchHits['hits']):
                print("%s\t%s\t\t%s\t%s" %
                      (idx + 1, hit['_score'], hit['_source']['id'], hit['_source']['title']))
            break

In [186]:
query = {      
    "query": {
        "multi_match": {
            "query": "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft",
            "fields": [f"title^1", f"data^1"]
         }
     }
}

In [187]:
search(query)

['15', '184', '858', '51', '102', '202', '14', '52', '380', '859', '948', '285', '390', '391', '442', '497', '643', '856', '857', '877', '864', '658']
10
The MAP Score of the corpus on the given query is: 0.39107142857142857
The optimized weights are: 
Num	Relevance Score		 ID	 Title
1	207.9244		184	scale models for thermo-aeroelastic research .
2	188.16914		486	similarity laws for aerothermoelastic testing .
3	176.17871		13	similarity laws for stressing heated wings .
4	164.00404		1268	stable combustion of a high-velocity gas in a heated
boundary layer .
5	161.11797		12	some structural and aerelastic considerations of high
speed flight .
6	133.99417		51	theory of aircraft structural models subjected to aerodynamic
heating and external loads .
7	126.516014		878	experimental model techniques and equipment for flutter
investigations .
8	126.23116		14	piston theory - a new aerodynamic tool for the
aeroelastician .
9	111.33605		1361	large deflections of structures subjected to heating
and 

In [72]:
query = {      
    "query": {
        "multi_match": {
            "query": "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft",
            "fields": ["title^1", "data^10"]
         }
     }
}